<a href="https://colab.research.google.com/github/Andreylive/UAV-human-detection/blob/main/YOLO_v5_for_UAV_human_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!git pull

!git reset --hard 886f1c03d839575afecb059accf74296fad395b6

In [17]:
# Install dependencies
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

# Clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [17]:
# Extract dataset
!curl -L "https://app.roboflow.com/ds/YCsKLvAbzh?key=wauxyPlLFl" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [17]:
# Load YAML file with our data
%cat data.yaml

In [6]:
# Define number of classes based on YAML
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [17]:
# Show model configaration
%cat ./data.yaml

In [8]:
# Customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [17]:
# Define model structure

%%writetemplate ./data.yaml

# parameters
train : ./train
val : ./valid
names : ['Person']
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [17]:
%cat ./models/yolov5s.yaml

In [17]:
!pip install wandb -q

In [17]:
!git pull

In [17]:
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda:0")
else:
    print("GPU not available, using CPU instead")
    device = torch.device("cpu")

In [17]:
!export WANDB_API_KEY=742aac2df6aee3508d1fdb5ad95cb60e7ea2dbdd && python train.py --img 416 --batch 16 --epochs 100 --data './data.yaml' --cfg ./models/yolov5s.yaml --weights '' --name ./yolov5s_results --cache

In [17]:
img = mpimg.imread('./runs/train/yolov5s_results2/results.png')

plt.figure(figsize=(20, 10))
plt.imshow(img)
plt.show()

In [ ]:
!python detect.py --weights ./runs/train/yolov5s_results2/weights/best.pt --img 416 --conf 0.4 --source ./valid/images

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
for det_img in [
    'train_BLA_0006_JPG.rf.586dd34289f6bc89c3fd33b16c30190e.jpg',
    'train_BRA_1003_JPG.rf.a3f32a60314fdb79d054f284d712f1c8.jpg',
    'train_ZRI_2012_JPG.rf.c9d7bfac5fbc26c34c2e567ee09ed736.jpg',
    'train_ZRI_2018_JPG.rf.20a15d8fc0c97f8f4add326d3890030c.jpg']:
    # load image examle
    img = mpimg.imread('/kaggle/working/yolov5/runs/detect/exp6/'+det_img)

    # Show image example
    plt.imshow(img)
    plt.show()

In [ ]:
!python val.py --weights ./runs/train/yolov5s_results2/weights/best.pt --data ./data.yaml --img 416 --half

In [ ]:
!ls /runs/val/exp

In [ ]:
img = mpimg.imread('./runs/val/exp/confusion_matrix.png')

plt.figure(figsize=(20, 10))
plt.imshow(img)
plt.show()

In [ ]:
import shutil

shutil.move('source_path/filename.extension', '/kaggle/working/filename.extension')

In [ ]:
!tar -czf output.tar.gz /kaggle/working/*